<a href="https://colab.research.google.com/github/chaerui7967/stock_predict_news_and_youtube/blob/master/%ED%95%98%EC%9D%B4%EB%8B%89%EC%8A%A4_%ED%86%B5%ED%95%A9%ED%85%8C%EC%9D%B4%EB%B8%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 오늘 시작

## 라이브러리 설치

In [1]:
!pip install finance-datareader
!pip install pykrx
!pip install pymysql
!pip install konlpy
from konlpy.tag import Okt
okt = Okt()

     |████████████████████████████████| 81 kB 3.9 MB/s 
     |████████████████████████████████| 60 kB 7.0 MB/s 
     |████████████████████████████████| 251 kB 33.1 MB/s 
     |████████████████████████████████| 43 kB 1.1 MB/s 
     |████████████████████████████████| 19.4 MB 7.2 MB/s 
     |████████████████████████████████| 86 kB 5.2 MB/s 
     |████████████████████████████████| 448 kB 34.0 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


## 패키지 & 함수 로딩

In [2]:
# 필요 라이브러리
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# 주식가격 로딩
import FinanceDataReader as fdr
from pykrx import stock

import datetime

In [3]:
def money_sur(stock_name, start, end):

    start_date = start
    end_date = end
    sample_code = stock_name

    dataset = fdr.DataReader(sample_code, start = start_date, end = end_date )
    dataset = dataset.reset_index()
    df = stock.get_market_fundamental_by_date(fromdate=start_date, todate=end_date, ticker=sample_code)
    df = df.reset_index()
    df.columns = ['Date','BPS', 'PER', 'PBR', 'EPS', 'DIV', 'DPS']
    data = pd.merge(dataset, df, on='Date')

    return data

In [4]:
def reverse_min_max_scaling(org_x, x): #종가 예측값
    org_x_np = np.asarray(org_x) 
    x_np = np.asarray(x)
    return (x_np * (org_x_np.max() - org_x_np.min())) + org_x_np.min()

## 통합테이블 생성

In [6]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
# total_table = pd.read_csv('/content/drive/My Drive/Final PJT - 업빛투/total_table.csv') # 기존 테이블

In [8]:
# 구성
dic = {'code' : [],
       'date' : [],
       'lstm' : [],
       'arima' : [],
       'fbprophet' : [],
       'RL' : [],
    #    'mail_news' : [],
    #    'mail_news_nsi' : [],
    #    'asia_news' : [],
    #    'asia_news_nsi': [],
    #    'youtube_sam' : [],
    #    'youtube_su' : [],
    #    'youtube_han' : [],
       'close' : []}

total_table = pd.DataFrame(dic)

In [9]:
total_table.columns

Index(['code', 'date', 'lstm', 'arima', 'fbprophet', 'RL', 'close'], dtype='object')

In [10]:
total_table

,code,date,lstm,arima,fbprophet,RL,close


## LSTM 예측결과

### 사용 데이터 로드 및 전처리

In [11]:
start_date = (datetime.datetime.now() - datetime.timedelta(40)).strftime("%Y%m%d")
end_date = datetime.datetime.now().strftime("%Y%m%d")
lstm = money_sur('000660', start_date, end_date)

In [12]:
lstm = lstm.set_index('Date')

In [13]:
lstm.head()

,Open,High,Low,Close,Volume,Change,BPS,PER,PBR,EPS,DIV,DPS
Date,,,,,,,,,,,,
2021-08-23,104500,105000,103000,103000,4515699,0.004878,71275,14.82,1.45,6952,1.14,1170
2021-08-24,104500,105500,104000,105000,4454743,0.019417,71275,15.10,1.47,6952,1.11,1170
2021-08-25,106000,106000,102500,103500,3194555,-0.014286,71275,14.89,1.45,6952,1.13,1170
2021-08-26,106000,106500,103500,104000,3694741,0.004831,71275,14.96,1.46,6952,1.13,1170
2021-08-27,103500,104500,101000,103500,3507782,-0.004808,71275,14.89,1.45,6952,1.13,1170


In [14]:
# 스케일링
from sklearn.preprocessing import MinMaxScaler

sc = MinMaxScaler()
lstm_sc = sc.fit_transform(lstm)

In [15]:
scale_cols = lstm.columns
lstm_sc_df = pd.DataFrame(lstm_sc, columns=[scale_cols], index=lstm.index)

In [16]:
lstm_sc_df.head()

,Open,High,Low,Close,Volume,Change,BPS,PER,PBR,EPS,DIV,DPS
Date,,,,,,,,,,,,
2021-08-23,0.567901,0.461538,0.492754,0.3750,0.594934,0.606340,0.0,0.379310,0.416667,0.0,0.666667,0.0
2021-08-24,0.567901,0.538462,0.637681,0.6250,0.581502,0.834172,0.0,0.620690,0.583334,0.0,0.333333,0.0
2021-08-25,0.753086,0.615385,0.420290,0.4375,0.303819,0.306045,0.0,0.439655,0.416667,0.0,0.555556,0.0
2021-08-26,0.753086,0.692308,0.565217,0.5000,0.414035,0.605602,0.0,0.500000,0.500000,0.0,0.555556,0.0
2021-08-27,0.444444,0.384615,0.202899,0.4375,0.372839,0.454565,0.0,0.439655,0.416667,0.0,0.555556,0.0


In [17]:
for i in scale_cols:
  for s in range(1, 11):  
    lstm_sc_df[(i+'shift_{}'.format(s),)] = lstm_sc_df[(i,)].shift(s)

In [18]:
lstm_sc_df.dropna()

,Open,High,Low,Close,Volume,Change,BPS,PER,PBR,EPS,DIV,DPS,Openshift_1,Openshift_2,Openshift_3,Openshift_4,Openshift_5,Openshift_6,Openshift_7,Openshift_8,Openshift_9,Openshift_10,Highshift_1,Highshift_2,Highshift_3,Highshift_4,Highshift_5,Highshift_6,Highshift_7,Highshift_8,Highshift_9,Highshift_10,Lowshift_1,Lowshift_2,Lowshift_3,Lowshift_4,Lowshift_5,Lowshift_6,Lowshift_7,Lowshift_8,...,PBRshift_1,PBRshift_2,PBRshift_3,PBRshift_4,PBRshift_5,PBRshift_6,PBRshift_7,PBRshift_8,PBRshift_9,PBRshift_10,EPSshift_1,EPSshift_2,EPSshift_3,EPSshift_4,EPSshift_5,EPSshift_6,EPSshift_7,EPSshift_8,EPSshift_9,EPSshift_10,DIVshift_1,DIVshift_2,DIVshift_3,DIVshift_4,DIVshift_5,DIVshift_6,DIVshift_7,DIVshift_8,DIVshift_9,DIVshift_10,DPSshift_1,DPSshift_2,DPSshift_3,DPSshift_4,DPSshift_5,DPSshift_6,DPSshift_7,DPSshift_8,DPSshift_9,DPSshift_10
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-09-06,0.814815,0.769231,0.782609,0.7500,0.016050,0.383453,0.0,0.750000,0.750000,0.0,0.222222,0.0,0.814815,1.000000,0.753086,0.444444,0.629630,0.444444,0.753086,0.753086,0.567901,0.567901,0.846154,1.000000,0.923077,0.692308,0.538462,0.384615,0.692308,0.615385,0.538462,0.461538,0.927536,0.927536,0.710145,0.347826,0.565217,0.202899,0.565217,0.420290,...,0.833333,0.750000,1.000000,0.750000,0.416667,0.416667,0.500000,0.416667,0.583334,0.416667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.111111,0.222222,0.000000,0.222222,0.555556,0.555556,0.555556,0.555556,0.333333,0.666667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2021-09-07,0.691358,0.615385,0.637681,0.5625,0.161277,0.308157,0.0,0.560345,0.583334,0.0,0.444444,0.0,0.814815,0.814815,1.000000,0.753086,0.444444,0.629630,0.444444,0.753086,0.753086,0.567901,0.769231,0.846154,1.000000,0.923077,0.692308,0.538462,0.384615,0.692308,0.615385,0.538462,0.782609,0.927536,0.927536,0.710145,0.347826,0.565217,0.202899,0.565217,...,0.750000,0.833333,0.750000,1.000000,0.750000,0.416667,0.416667,0.500000,0.416667,0.583334,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.222222,0.111111,0.222222,0.000000,0.222222,0.555556,0.555556,0.555556,0.555556,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2021-09-08,0.506173,0.846154,0.565217,0.7500,0.236989,0.754829,0.0,0.750000,0.750000,0.0,0.222222,0.0,0.691358,0.814815,0.814815,1.000000,0.753086,0.444444,0.629630,0.444444,0.753086,0.753086,0.615385,0.769231,0.846154,1.000000,0.923077,0.692308,0.538462,0.384615,0.692308,0.615385,0.637681,0.782609,0.927536,0.927536,0.710145,0.347826,0.565217,0.202899,...,0.583334,0.750000,0.833333,0.750000,1.000000,0.750000,0.416667,0.416667,0.500000,0.416667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.444444,0.222222,0.111111,0.222222,0.000000,0.222222,0.555556,0.555556,0.555556,0.555556,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2021-09-09,0.629630,0.615385,0.492754,0.3750,0.312293,0.086413,0.0,0.379310,0.416667,0.0,0.666667,0.0,0.506173,0.691358,0.814815,0.814815,1.000000,0.753086,0.444444,0.629630,0.444444,0.753086,0.846154,0.615385,0.769231,0.846154,1.000000,0.923077,0.692308,0.538462,0.384615,0.692308,0.565217,0.637681,0.782609,0.927536,0.927536,0.710145,0.347826,0.565217,...,0.750000,0.583334,0.750000,0.833333,0.750000,1.000000,0.750000,0.416667,0.416667,0.500000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.222222,0.444444,0.222222,0.111111,0.222222,0.000000,0.222222,0.555556,0.555556,0.555556,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2021-09-10,0.444444,0.538462,0.492754,0.6250,0.040459,0.834172,0.0,0.620690,0.583334,0.0,0.333333,0.0,0.629630,0.506173,0.691358,0.814815,0.814815,1.000000,0.753086,0.444444,0.629630,0.444444,0.615385,0.846154,0.615385,0.769231,0.846154,1.000000,0.923077,0.692308,0.538462,0.384615,0.492754,0.565217,0.637681,0.782609,0.927536,0.927536,0.710145,0.347826,...,0.416667,0.750000,0.583334,0.750000,0.833333,0.750000,1.000000,0.750000,0.416667,0.416667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.666667,0.222222,0.444444,0.222222,0.111111,0.222222,0.000000,0.222222,0.555556,0.555556,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [19]:
X_test = lstm_sc_df.dropna().drop(columns=scale_cols, axis=1)
y_test = lstm_sc_df.dropna()[['Close']]

X_test= X_test.values
y_test = y_test.values

In [20]:
X_test

array([[0.81481481, 1.        , 0.75308642, ..., 0.        , 0.        ,
        0.        ],
       [0.81481481, 0.81481481, 1.        , ..., 0.        , 0.        ,
        0.        ],
       [0.69135802, 0.81481481, 0.81481481, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.62962963, 0.50617284, 0.75308642, ..., 0.        , 0.        ,
        0.        ],
       [0.13580247, 0.62962963, 0.50617284, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.13580247, 0.62962963, ..., 0.        , 0.        ,
        0.        ]])

In [21]:
X_test_t = X_test.reshape(X_test.shape[0], 10, 12)

### 모델 로드

In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv1D, Lambda,Dropout, Flatten
from tensorflow.keras.losses import Huber
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

model = Sequential()

model.add(LSTM(64,
               input_shape = (10, 12),
               return_sequences= True))
model.add(LSTM(64,
               return_sequences = False))
model.add(Dense(32, activation = 'tanh'))
model.add(Flatten())
model.add(Dense(1))


lstm_path = '/content/drive/My Drive/Final PJT - 업빛투/Regression/LSTM/'
hy_lstm = 'lstm_hy.h5'

model.load_weights(lstm_path+hy_lstm)

### 모델 결과

In [23]:
pred = model.predict(X_test_t)

In [24]:
pred.shape, y_test.shape

((17, 1), (17, 1))

In [25]:
pred_act = reverse_min_max_scaling(lstm['Close'], pred)

In [26]:
total_table['date'] = lstm.index[-10:]
total_table['lstm'] = pred_act[-10:]
total_table['close'] = lstm.Close[-10:].values.reshape(-1,1)
total_table['code'] = '005930'
total_table['change'] = lstm.Change[-10:].values.reshape(-1,1)

In [27]:
total_table

,code,date,lstm,arima,fbprophet,RL,close,change
0,005930,2021-09-15,107611.110840,NaN,NaN,NaN,107500,0.000000
1,005930,2021-09-16,106315.300781,NaN,NaN,NaN,104000,-0.032558
2,005930,2021-09-17,106507.104492,NaN,NaN,NaN,107000,0.028846
3,005930,2021-09-23,105733.528809,NaN,NaN,NaN,105500,-0.014019
4,005930,2021-09-24,105307.092773,NaN,NaN,NaN,104000,-0.014218
5,005930,2021-09-27,105843.469727,NaN,NaN,NaN,104500,0.004808
6,005930,2021-09-28,104409.901855,NaN,NaN,NaN,103500,-0.009569
7,005930,2021-09-29,105048.020996,NaN,NaN,NaN,100000,-0.033816
8,005930,2021-09-30,101771.489502,NaN,NaN,NaN,103000,0.030000
9,005930,2021-10-01,101373.722412,NaN,NaN,NaN,100000,-0.029126


## arima 예측결과

In [28]:
end_date = datetime.datetime.now().strftime("%Y%m%d")
samsung_df = money_sur('000660', '2018', end_date)

In [29]:
df = samsung_df[['Date','Close']]
df.columns = ['Date', 'Price']

In [30]:
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm
# 학습, 테스트 데이터 분리
for ru in range(-10,0):
    rus = ru+1
    train = df[:ru]
    test = df[ru:]

    model = ARIMA(train.Price.values, order = (2,1,2))
    model_fit = model.fit(trend = 'nc', full_output = True, disp = True)

    fore = model_fit.forecast(steps=1) # 오늘 예측

    # 오늘 예측 데이터
    pred_arima_y = fore[0].tolist()

    # 실제 오늘 데이터 
    test_y = test.Price.values

    # # 신뢰구간
    # # 예측 데이터 최소값
    # pred_y_lower = []
    # # 예측 데이터 최대값
    # pred_y_upper = []

    # for lower_upper in fore[2]:
    #     lower = lower_upper[0]
    #     upper = lower_upper[1]
    #     pred_y_lower.append(lower)
    #     pred_y_upper.append(upper)
    if rus == 0:
        total_table['arima'].iloc[ru:] = pred_arima_y
    else:
        total_table['arima'].iloc[ru:rus] = pred_arima_y

print(model_fit.summary())

                             ARIMA Model Results                              
Dep. Variable:                    D.y   No. Observations:                  922
Model:                 ARIMA(2, 1, 2)   Log Likelihood               -8381.695
Method:                       css-mle   S.D. of innovations           2146.965
Date:                Sat, 02 Oct 2021   AIC                          16773.390
Time:                        08:26:41   BIC                          16797.522
Sample:                             1   HQIC                         16782.598
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1.D.y      0.5201      0.206      2.521      0.012       0.116       0.925
ar.L2.D.y     -0.6674      0.129     -5.186      0.000      -0.920      -0.415
ma.L1.D.y     -0.5612      0.187     -3.005      0.0

In [31]:
total_table

,code,date,lstm,arima,fbprophet,RL,close,change
0,005930,2021-09-15,107611.110840,107531.570906,NaN,NaN,107500,0.000000
1,005930,2021-09-16,106315.300781,107757.076005,NaN,NaN,104000,-0.032558
2,005930,2021-09-17,106507.104492,104246.157040,NaN,NaN,107000,0.028846
3,005930,2021-09-23,105733.528809,106592.234904,NaN,NaN,105500,-0.014019
4,005930,2021-09-24,105307.092773,105347.857731,NaN,NaN,104000,-0.014218
5,005930,2021-09-27,105843.469727,104174.314218,NaN,NaN,104500,0.004808
6,005930,2021-09-28,104409.901855,104586.031889,NaN,NaN,103500,-0.009569
7,005930,2021-09-29,105048.020996,103492.603294,NaN,NaN,100000,-0.033816
8,005930,2021-09-30,101771.489502,99993.489997,NaN,NaN,103000,0.030000
9,005930,2021-10-01,101373.722412,102619.708155,NaN,NaN,100000,-0.029126


## fbprophet 예측

In [32]:
end_date = datetime.datetime.now().strftime("%Y%m%d")
samsung_df = money_sur('000660', start = '20180101', end = end_date)
df1 = samsung_df[['Date','Close']]
df1.columns = ['ds', 'y']

from fbprophet import Prophet

prophet = Prophet(seasonality_mode = 'multiplicative',
                 yearly_seasonality=True, 
                 weekly_seasonality=True,
                 daily_seasonality=True,
                 changepoint_prior_scale=0.6)

prophet.fit(df1)

# 10일단위로 예측값을 가져옴
future_data = prophet.make_future_dataframe(periods = 10, freq = 'd')
forecast_data = prophet.predict(future_data)

In [33]:
list(total_table.date.values)

[numpy.datetime64('2021-09-15T00:00:00.000000000'),
 numpy.datetime64('2021-09-16T00:00:00.000000000'),
 numpy.datetime64('2021-09-17T00:00:00.000000000'),
 numpy.datetime64('2021-09-23T00:00:00.000000000'),
 numpy.datetime64('2021-09-24T00:00:00.000000000'),
 numpy.datetime64('2021-09-27T00:00:00.000000000'),
 numpy.datetime64('2021-09-28T00:00:00.000000000'),
 numpy.datetime64('2021-09-29T00:00:00.000000000'),
 numpy.datetime64('2021-09-30T00:00:00.000000000'),
 numpy.datetime64('2021-10-01T00:00:00.000000000')]

In [34]:
total_table['fbprophet'] = forecast_data[forecast_data['ds'].isin(list(total_table.date.values))]['yhat'].values

In [35]:
total_table

,code,date,lstm,arima,fbprophet,RL,close,change
0,005930,2021-09-15,107611.110840,107531.570906,105170.103805,NaN,107500,0.000000
1,005930,2021-09-16,106315.300781,107757.076005,105230.196998,NaN,104000,-0.032558
2,005930,2021-09-17,106507.104492,104246.157040,105085.161569,NaN,107000,0.028846
3,005930,2021-09-23,105733.528809,106592.234904,104010.986018,NaN,105500,-0.014019
4,005930,2021-09-24,105307.092773,105347.857731,103765.708080,NaN,104000,-0.014218
5,005930,2021-09-27,105843.469727,104174.314218,102621.668387,NaN,104500,0.004808
6,005930,2021-09-28,104409.901855,104586.031889,102742.237966,NaN,103500,-0.009569
7,005930,2021-09-29,105048.020996,103492.603294,102328.026323,NaN,100000,-0.033816
8,005930,2021-09-30,101771.489502,99993.489997,102223.087220,NaN,103000,0.030000
9,005930,2021-10-01,101373.722412,102619.708155,101927.313323,NaN,100000,-0.029126


In [36]:
# total_table.to_csv('./total_table.csv', index =False)

## RL 결과

### 데이터 로딩 및 함수 로딩

In [37]:
import time
import copy
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F

# 금융정보 조회하기
def dataset_loaderKR(stock_name, start, end, train_ratio=0.8):
    start_date = start
    end_date = end
    sample_code = stock_name

    dataset = fdr.DataReader(sample_code, start = start_date, end = end_date )
    dataset = dataset.reset_index()
    df = stock.get_market_fundamental_by_date(fromdate=start_date, todate=end_date, ticker=sample_code)
    df = df.reset_index()
    df.columns = ['Date','BPS', 'PER', 'PBR', 'EPS', 'DIV', 'DPS']
    data = pd.merge(dataset, df, on='Date')
    date_split = data.index[int(train_ratio*len(data))] 

    return data[:date_split], data[date_split:], date_split


# ENV 설정
class Environment1:
    def __init__(self, data, history_t=90):
        self.data = data
        self.history_t = history_t
        self.reset()
        
    def reset(self):
        self.t = 0
        self.done = False
        self.profits = 0
        self.positions = []
        self.position_value = 0
        self.history = [0 for _ in range(self.history_t)]
        return [self.position_value] + self.history # obs

    def step(self, act):
        reward = 0
        
        # action
        # 0: Idle
        # 1: 매수
        # 2: 매도
        if act == 1: #매수
            self.positions.append(self.data.iloc[self.t, :]['Close'])
        elif act == 2: # 매도
            if len(self.positions) == 0:
                reward = -1
            else:
                profits = 0
                for p in self.positions:
                    profits += (self.data.iloc[self.t, :]['Close'] - p)
                reward += profits
                self.profits += profits
                self.positions = []
        
        # set next time
        self.t += 1
        
        self.position_value = 0
        for p in self.positions:
            self.position_value += (self.data.iloc[self.t, :]['Close'] - p)
        self.history.pop(0)
        self.history.append(self.data.iloc[self.t, :]['Close'] - self.data.iloc[(self.t-1), :]['Close'])
        if (self.t==len(self.data)-1):
            self.done=True
        # clipping reward
        if reward > 0:
            reward = 1
        elif reward < 0:
            reward = -1
        #print ("t={%d}, done={%str}"%(self.t,self.done))
        return [self.position_value] + self.history, reward, self.done # obs, reward, done

# MODEL
class Q_Network(nn.Module):
    def __init__(self,obs_len,hidden_size,actions_n):
        super(Q_Network,self).__init__()
        self.fc_val = nn.Sequential(
            nn.Linear(obs_len, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, actions_n)
        )
    def forward(self,x):
        h = self.fc_val(x)
        return (h)

In [38]:
(train, test, date_split) = dataset_loaderKR('000660', start = '20180101', end = end_date)

### 모델 로딩 및 결과

In [44]:
rl_path = '/content/drive/My Drive/Final PJT - 업빛투/Regression/RL/하이닉스_결과/'
hy_rl = 'hynix_rl_재무정보.h5'

In [45]:
# 모델 로딩
model = torch.load(rl_path+hy_rl)

test_env = Environment1(test)
pobs = test_env.reset()
test_acts = []
test_rewards = []

for _ in range(len(test_env.data)-1):
    pact = model(torch.from_numpy(np.array(pobs, dtype=np.float32).reshape(1, -1)))
    pact = np.argmax(pact.data)
    test_acts.append(pact.item())
            
    obs, reward, done = test_env.step(pact.numpy())
    test_rewards.append(reward)

    pobs = obs

# sell remaining stocks
profits = 0
for p in test_env.positions:
	profits += (test_env.data.iloc[test_env.t, :]['Close'] - p)

test_env.profits += profits
test_env.positions = []

test_profits = test_env.profits
pact = model(torch.from_numpy(np.array(pobs, dtype=np.float32).reshape(1, -1)))
pact = np.argmax(pact.data)
next_action = pact.item()
test['Action'] = test_acts + [next_action]


print( )
print( test )
print( )
print("결과")
print( "손익결과 $%.2f" % round(test_profits,2) )
# 0: Idle
# 1: 매수
# 2: 매도


          Date    Open    High     Low   Close  ...   PBR   EPS   DIV   DPS  Action
739 2021-01-05  124500  132500  124000  130500  ...  1.98  2943  0.77  1000       1
740 2021-01-06  132500  137000  130000  131000  ...  1.99  2943  0.76  1000       2
741 2021-01-07  132500  137500  131500  134500  ...  2.04  2943  0.74  1000       2
742 2021-01-08  136000  138000  132500  138000  ...  2.10  2943  0.72  1000       2
743 2021-01-11  138000  140000  131000  133000  ...  2.02  2943  0.75  1000       1
..         ...     ...     ...     ...     ...  ...   ...   ...   ...   ...     ...
919 2021-09-27  104000  105500  103500  104500  ...  1.47  6952  1.12  1170       1
920 2021-09-28  105000  105500  103000  103500  ...  1.45  6952  1.13  1170       2
921 2021-09-29  101000  102000   99700  100000  ...  1.40  6952  1.17  1170       0
922 2021-09-30   99900  103500   99600  103000  ...  1.45  6952  1.14  1170       0
923 2021-10-01  102000  102500  100000  100000  ...  1.40  6952  1.17  1170

In [46]:
test['Action'] = np.where(test['Action'] == 1,1,
                          np.where(test['Action'] == 2,-1,0))

In [47]:
test['Action'][-10:]

914   -1
915   -1
916    0
917   -1
918    1
919    1
920   -1
921    0
922    0
923    0
Name: Action, dtype: int64

In [48]:
total_table['RL'] = test[test['Date'].isin(list(total_table.date.values))]['Action'].values

In [49]:
total_table

,code,date,lstm,arima,fbprophet,RL,close,change
0,005930,2021-09-15,107611.110840,107531.570906,105170.103805,-1,107500,0.000000
1,005930,2021-09-16,106315.300781,107757.076005,105230.196998,-1,104000,-0.032558
2,005930,2021-09-17,106507.104492,104246.157040,105085.161569,0,107000,0.028846
3,005930,2021-09-23,105733.528809,106592.234904,104010.986018,-1,105500,-0.014019
4,005930,2021-09-24,105307.092773,105347.857731,103765.708080,1,104000,-0.014218
5,005930,2021-09-27,105843.469727,104174.314218,102621.668387,1,104500,0.004808
6,005930,2021-09-28,104409.901855,104586.031889,102742.237966,-1,103500,-0.009569
7,005930,2021-09-29,105048.020996,103492.603294,102328.026323,0,100000,-0.033816
8,005930,2021-09-30,101771.489502,99993.489997,102223.087220,0,103000,0.030000
9,005930,2021-10-01,101373.722412,102619.708155,101927.313323,0,100000,-0.029126


## News 긍부정 결과
- '매일경제', '아시아경제'

In [50]:
# corp_list = ['samsung', 'hyundai', 'lg', 'sk', 'celltrion'] 참고용
stock_num_list = ['005930', '005380', '051910', '000660', '068270']
news = ['maeil_news_craw', 'asia_news_craw']
start = (datetime.datetime.now() - datetime.timedelta(20)).strftime("%Y-%m-%d").replace('-','')
end = datetime.datetime.now().strftime("%Y-%m-%d").replace('-', '')

In [51]:
# news_df = pd.read_csv('/content/drive/My Drive/Final PJT - 업빛투/감성분석/Final Data/1. samsung_score.csv') # 연습용

### 관련 함수 및 패키지 로드

In [52]:
import pymysql
import sys
import re

In [53]:
# 블로그에서 가져온 기본적인 한국어 긍부정 텍스트 목록
with open('/content/drive/My Drive/Final PJT - 업빛투/감성분석/data/positive_words_self.txt', encoding='utf-8') as pos_blog:
    positive_blog = pos_blog.readlines()
positive_blog = [pos_blog.replace('\n', '') for pos_blog in positive_blog]
with open('/content/drive/My Drive/Final PJT - 업빛투/감성분석/data/negative_words_self.txt', encoding='utf-8') as neg_blog:
    negative_blog = neg_blog.readlines()
negative_blog = [neg_blog.replace('\n', '') for neg_blog in negative_blog]

# KOSELF 감성 어휘 사전
with open('/content/drive/My Drive/Final PJT - 업빛투/감성분석/data/KOSELF_pos.txt', encoding='utf-8') as pos:
    positive = pos.readlines()
positive = [pos.replace('\n', '') for pos in positive]
with open('/content/drive/My Drive/Final PJT - 업빛투/감성분석/data/KOSELF_neg.txt', encoding='utf-8') as neg:
    negative = neg.readlines()
negative = [neg.replace('\n', '') for neg in negative]

In [54]:
def news_db(news, stock):
    db = pymysql.connect(
            user='root', 
            passwd='1234', 
            host='3.35.70.166', 
            db='proj', 
            charset = 'utf8'
        )

    cursor = db.cursor(pymysql.cursors.DictCursor)
    news1 = news + '_news_craw'

    sql = "select * from {0}_{1} where (length(date)=10) and (date between {2}00 and {3}23)".format(news1, stock, start, end)
    cursor.execute(sql)
    result = cursor.fetchall()

    # DataFrame으로 변경
    df = pd.DataFrame(result)

    db.close()

    df.rename(columns={'date': 'datetime'}, inplace=True)

    # DataFrame 형태를 통일하기 위해 date 컬럼 추가
    df['date'] = df['datetime'].str[0:4] + '-' + df['datetime'].str[4:6] + '-' + df['datetime'].str[6:8]
    df['date'] = pd.to_datetime(df['date'])

    # 결측치 제거 → 데이터 로드 시 완료했기 때문에 그다지 필요하지 않은 과정
    df = df.dropna()

    # 시간순으로 정렬
    df.sort_values('datetime', inplace=True)
    df.reset_index(inplace=True, drop=True)

    ### 4) Holidays
    db = pymysql.connect(user='root',
                         passwd='1234', 
                         host='3.35.70.166', 
                         db='proj',
                         charset='utf8')

    cursor = db.cursor(pymysql.cursors.DictCursor)

    # 4-1) 주말 및 공휴일 데이터
    sql = "select * from holidays"
    cursor.execute(sql)
    result = cursor.fetchall()

    holidays = pd.DataFrame(result)

    db.close()

    holidays['date'] = pd.to_datetime(holidays['date'])

    url = 'https://raw.githubusercontent.com/chaerui7967/stock_predict_news_and_youtube/master/Sentiment_Analysis/data/stopwords_ver1.txt'
    stopwords = list(pd.read_csv(url, header=None)[0])

    # Stop Words List에 각 매체명 추가
    except_media_list = ['매일경제', '매일', '경제', 'maeil', 'MK', 'mk',
                         '아시아경제', '아시아', 'Asia', 'ASIA', 'asia',
                         '삼프로TV', '삼프로', 'TV',
                         '슈카월드', '슈카', '월드'
                         '한국경제TV', '한국']
    for word in except_media_list:
        stopwords.append(word)
    
    #### 2. Preprocessing
    '''감성 어휘 사전 : negative / positive
       뉴스 데이터 : samsung / hyundai / lg / sk
       주식 데이터 : stock_samsung / stock_hyundai / stock_lg / stock_sk
       공휴일 데이터 : holidays'''
    
    
    ### 1) 뉴스 데이터 날짜 조정
    
    ## 1-1)업로드 시각 컬럼 추가
    df['time'] = df['datetime'].str[-2:]
    
    ## 1-2) 전일 15시 ~ 금일 15시로 날짜 조정
    after_market = ['15', '16', '17', '18', '19', '20', '21', '22', '23']

    for j in range(len(df['time'])):
        if df['time'][j] in after_market:
            df['date'][j] += datetime.timedelta(1)
        else:
            pass
    
    ## 1-3) 텍스트 전처리
    # \n, \t, \r 제거
    df['text'] = df['text'].str.replace('[\n|\t|\r]', '')
    df['text'] = df['text'].str.replace('\[|\]', '')
    df['text'] = df['text'].str.replace('\<.*\>', '')
    df['text'] = df['text'].str.replace('\\', '')
    df['text'] = df['text'].str.replace('이미지', '')


    ### 2) 주말 및 공휴일 제외
    
    ## 2-1) 주말 및 공휴일만 추출
    market_closed =holidays[holidays['holiday']=="O"].reset_index(drop=True)
    
    ## 2-3) 휴장일 List 생성
    market_closed_list = list(market_closed['date'])
    
    ## 2-4) iteration limit 조정
    limit_number = 15000
    sys.setrecursionlimit(limit_number)
    
    
    while len(df[df['date'].isin(market_closed_list)]['date']) != 0:
        for hoil in df[df['date'].isin(market_closed_list)]['date'].index:
            df['date'][hoil] += datetime.timedelta(1)
    
    ### 3) Tokenization 컬럼 추가
    
    df['Tokenization'] = 0
    rows = df.shape[0]
    for row in range(rows):
        hangeul = re.compile('[^ ㄱ-ㅣ 가-힣]')                                    # 정규 표현식 → 한글 추출 규칙 : 띄어쓰기(1개)를 포함한 한글
        result = hangeul.sub('', df['text'][row])   # 위에 설정한 hangeul 규칙을 text에 적용
        okt = Okt()                                                               # 형태소 추출
        nouns = okt.nouns(df['text'][row])
        nouns = [x for x in nouns if len(x) > 1]                                  # 한 글자 키워드 제거
        nouns = [x for x in nouns if x not in stopwords]                          # 불용어 제거
        
        corpus = " ".join(nouns)                                                  # List를 String으로 변환
        df['Tokenization'][row] = corpus


    # 점수 산출

    df['Positive_Score'] = 0
    df['Negative_Score'] = 0
    df['Ratio'] = 0.1
    df['Pred'] = 0
    df['NSI'] = 0.1
    
    for score in range(len(df)):
        pos_score = 0 ; neg_score = 0
        
        for token in range(len(df['Tokenization'][score].split())):
            if df['Tokenization'][score].split()[token] in positive:
                pos_score += 1
            elif df['Tokenization'][score].split()[token] in negative:
                neg_score += 1
            else:
                pass
        
        df['Positive_Score'][score] = pos_score
        df['Negative_Score'][score] = neg_score
        
        # 긍정과 부정의 비율
        if (pos_score==0) and (neg_score==0):
            df['Ratio'][score] = 0.5   # 둘 다 0일 경우에는 긍정으로 가정
        else:
            df['Ratio'][score] = pos_score / (pos_score + neg_score)
        
        # 예측 결과
        if df['Ratio'][score]>=0.5:
            df['Pred'][score] = 1
        else:
            df['Pred'][score] = -1
        
        # 뉴스심리지수(NSI) 계산
        if (pos_score==0) and (neg_score==0):
            df['NSI'][score] = 101
        else:
            df['NSI'][score] = (pos_score - neg_score) / (pos_score + neg_score) * 100 + 100
    
    return df

### 매일경제

In [55]:
# mail_df = news_df[(news_df['news'] == '매일경제') & (news_df['st_cd'] == 5930)]

In [56]:
mail_df = news_db('maeil', '000660')

In [57]:
mail_df.head()

,st_n,st_cd,news,datetime,title,url,text,date,time,Tokenization,Positive_Score,Negative_Score,Ratio,Pred,NSI
0,하이닉스,000660,매일경제,2021091504,"미래에셋증권, AI·2차전지 등 신성장株에 집중투자",http://news.mk.co.kr/newsRead.php?no=888707&ye...,◆ 추천상품 ◆ 최근 글로벌 산업 패러다임은 굴뚝산업에서 첨단 정보기술(IT)산업으...,2021-09-15,04,추천 상품 최근 글로벌 산업 패러다임 굴뚝 산업 첨단 정보기술 산업 점차 변화 인공...,3,0,1.000000,1,200.000000
1,하이닉스,000660,매일경제,2021091516,"강훈식 의원 ""韓기업 보호무역 예민…반도체 中공장 닫을 수 있다더라""",http://news.mk.co.kr/newsRead.php?no=892266&ye...,"강훈식 더불어민주당 의원이 14일(현지시간) 미국 워싱턴DC에서 ""한국 지상사 간담...",2021-09-16,16,강훈 민주당 의원 현지 미국 워싱턴 상사 간담 현장 기업 보호무역 보호 주의 더욱 ...,3,2,0.600000,1,120.000000
2,하이닉스,000660,매일경제,2021091517,"""대형주 너만은 믿었건만""…반도체 화학 이어 플랫폼 주가 급락에 동학개미 속탄다",http://news.mk.co.kr/newsRead.php?no=892687&ye...,"국내 증시를 대표하는 반도체, 2차전지 등 주요 업종 시가총액 상위 대장주를 중심...",2021-09-16,17,국내 증시 대표 반도체 주요 업종 시가총액 상위 장주 중심 주가 급락 올해 등락 마...,2,13,0.133333,-1,26.666667
3,하이닉스,000660,매일경제,2021091614,"고려대의료원, 아프간 특별기여자 전격 의료지원",http://news.mk.co.kr/newsRead.php?no=895839&ye...,고려대 안암병원 치과 최연조 교수가 아프간 어린이를 진료하고 있다. 고려대의료원(의...,2021-09-16,14,고려대 안암 병원 치과 최연 교수 아프간 어린이 진료 고려대 의료 의무 부총 의료 ...,5,0,1.000000,1,200.000000
4,하이닉스,000660,매일경제,2021091619,"네이버 오르는데 왜 카카오는…154만 개미주주 속 탄다, 8일간 21% 털썩",http://news.mk.co.kr/newsRead.php?no=897323&ye...,경기 성남시 카카오 판교오피스. 한주형 기자 정부와 금융당국의 빅테크 규제로 인해 ...,2021-09-17,19,경기 성남시 카카오 판교 오피스 한주 기자 정부 금융 당국 테크 규제 연일 주가 급...,0,5,0.000000,-1,0.000000


In [58]:
mail_ratio = mail_df[mail_df['date'].isin(list(total_table.date.values))][['date','Ratio']]
mail_nsi = mail_df[mail_df['date'].isin(list(total_table.date.values))][['date','NSI']]

In [59]:
mail_ratio = mail_ratio.groupby(['date']).mean().reset_index()
mail_nsi = mail_nsi.groupby(['date']).mean().reset_index()

In [60]:
mail_ratio.columns = ['date', 'mail_news']
mail_nsi.columns = ['date', 'mail_news_nsi']

In [61]:
total_table = pd.merge(total_table, mail_ratio, on='date', how='left')
total_table = pd.merge(total_table, mail_nsi, on='date', how='left')

In [62]:
total_table = total_table[['code', 'date', 'lstm', 'arima', 'fbprophet', 'RL', 'mail_news', 'mail_news_nsi', 'close', 'change']]

In [63]:
total_table

,code,date,lstm,arima,fbprophet,RL,mail_news,mail_news_nsi,close,change
0,005930,2021-09-15,107611.110840,107531.570906,105170.103805,-1,1.000000,200.000000,107500,0.000000
1,005930,2021-09-16,106315.300781,107757.076005,105230.196998,-1,0.577778,115.555556,104000,-0.032558
2,005930,2021-09-17,106507.104492,104246.157040,105085.161569,0,0.500000,100.000000,107000,0.028846
3,005930,2021-09-23,105733.528809,106592.234904,104010.986018,-1,0.500000,100.333333,105500,-0.014019
4,005930,2021-09-24,105307.092773,105347.857731,103765.708080,1,0.186667,37.333333,104000,-0.014218
5,005930,2021-09-27,105843.469727,104174.314218,102621.668387,1,0.357143,71.428571,104500,0.004808
6,005930,2021-09-28,104409.901855,104586.031889,102742.237966,-1,0.403175,80.801587,103500,-0.009569
7,005930,2021-09-29,105048.020996,103492.603294,102328.026323,0,0.357143,71.428571,100000,-0.033816
8,005930,2021-09-30,101771.489502,99993.489997,102223.087220,0,0.200000,40.000000,103000,0.030000
9,005930,2021-10-01,101373.722412,102619.708155,101927.313323,0,0.500000,100.000000,100000,-0.029126


### 아시아

In [64]:
# asia_df = news_df[(news_df['news'] == '아시아경제') & (news_df['st_cd'] == 5930)]

In [65]:
asia_df = news_db('asia', '000660')

In [66]:
asia_df.tail()

,st_n,st_cd,news,datetime,title,url,text,date,time,Tokenization,Positive_Score,Negative_Score,Ratio,Pred,NSI
73,하이닉스,000660,아시아경제,2021100109,"코스피, 간밤 美증시 하락세 영향에 장초반 약세",https://view.asiae.co.kr/article/2021100109333...,썝蹂몃낫湲 븘씠肄아시아경제 박지환 기자 간밤 미국 증시가 인플레이션 우려로 일제히 ...,2021-10-01,09,몃낫 븘씠 박지환 기자 간밤 미국 증시 인플레이션 우려 일제 하락 마감 국내 증시 ...,1,14,0.066667,-1,13.333333
74,하이닉스,000660,아시아경제,2021100110,"에스엠코어, SK하이닉스와 M16 AGV 시스템 공급계약 체결",https://view.asiae.co.kr/article/2021100110082...,"아시아경제 이민지 기자 에스엠코어에스엠코어007820코스닥증권정보현재가7,450전일...",2021-10-01,10,이민지 기자 에스 엠코 에스 엠코 코스닥 증권 정보 현재 전일 대비 거래 전일 장마...,0,0,0.500000,1,101.000000
75,하이닉스,000660,아시아경제,2021100111,"코스피, 1.42% 하락 '3020선' 후퇴...코스닥은 장중 1000선 붕괴",https://view.asiae.co.kr/article/2021100111165...,코스피 지수가 미국 증시 부진 여파에 하락 출발한 1일 서울 을지로 하나은행 딜링룸...,2021-10-01,11,코스피 지수 미국 증시 부진 여파 하락 출발 서울 을지로 하나은행 딜링룸 딜러 업무...,1,14,0.066667,-1,13.333333
76,하이닉스,000660,아시아경제,2021100114,"코스피, 외국인·기관 팔자에 '3010선' 뚝...""이달 중순까지 인내 시간 필요""",https://view.asiae.co.kr/article/2021100114210...,썝蹂몃낫湲 븘씠肄아시아경제 박지환 기자 코스피가 외국인과 기관의 동반 순매도에 1....,2021-10-01,14,몃낫 븘씠 박지환 기자 코스피 외국인 기관 동반 매도 하락 보이 오후 기준 코스피 ...,3,15,0.166667,-1,33.333333
77,하이닉스,000660,아시아경제,2021100116,코스피 3000선 위협...코스닥도 천스닥 아래로,https://view.asiae.co.kr/article/2021100116073...,코스피 지수가 미국 증시 부진 여파에 하락 출발한 1일 서울 을지로 하나은행 딜링룸...,2021-10-05,16,코스피 지수 미국 증시 부진 여파 하락 출발 서울 을지로 하나은행 딜링룸 딜러 업무...,1,21,0.045455,-1,9.090909


In [67]:
asia_ratio = asia_df[asia_df['date'].isin(list(total_table.date.values))][['date','Ratio']]
asia_nsi = asia_df[asia_df['date'].isin(list(total_table.date.values))][['date','NSI']]

asia_ratio = asia_ratio.groupby(['date']).mean().reset_index()
asia_nsi = asia_nsi.groupby(['date']).mean().reset_index()

asia_ratio.columns = ['date', 'asia_news']
asia_nsi.columns = ['date', 'asia_news_nsi']

In [68]:
total_table = pd.merge(total_table, asia_ratio, on='date', how='left')
total_table = pd.merge(total_table, asia_nsi, on='date', how='left')

In [69]:
# 컬럼 리셋
total_table = total_table[['code', 'date', 'lstm', 'arima', 'fbprophet', 'RL', 'mail_news', 'mail_news_nsi', 'asia_news', 'asia_news_nsi', 'close', 'change']]

In [70]:
total_table

,code,date,lstm,arima,fbprophet,RL,mail_news,mail_news_nsi,asia_news,asia_news_nsi,close,change
0,005930,2021-09-15,107611.110840,107531.570906,105170.103805,-1,1.000000,200.000000,0.409001,81.800144,107500,0.000000
1,005930,2021-09-16,106315.300781,107757.076005,105230.196998,-1,0.577778,115.555556,0.789286,157.857143,104000,-0.032558
2,005930,2021-09-17,106507.104492,104246.157040,105085.161569,0,0.500000,100.000000,0.305641,61.128205,107000,0.028846
3,005930,2021-09-23,105733.528809,106592.234904,104010.986018,-1,0.500000,100.333333,0.471347,94.519481,105500,-0.014019
4,005930,2021-09-24,105307.092773,105347.857731,103765.708080,1,0.186667,37.333333,0.457738,91.547619,104000,-0.014218
5,005930,2021-09-27,105843.469727,104174.314218,102621.668387,1,0.357143,71.428571,0.525683,105.367427,104500,0.004808
6,005930,2021-09-28,104409.901855,104586.031889,102742.237966,-1,0.403175,80.801587,0.282062,56.537338,103500,-0.009569
7,005930,2021-09-29,105048.020996,103492.603294,102328.026323,0,0.357143,71.428571,0.350000,70.000000,100000,-0.033816
8,005930,2021-09-30,101771.489502,99993.489997,102223.087220,0,0.200000,40.000000,0.000000,0.000000,103000,0.030000
9,005930,2021-10-01,101373.722412,102619.708155,101927.313323,0,0.500000,100.000000,0.200000,40.250000,100000,-0.029126


In [71]:
# total_table.to_csv('total_table.csv', index=False)

## youtube 긍부정 결과
- ['한국경제TV', '슈카월드', '삼프로TV_경제의신과함께']

In [72]:
# # 이전 라벨링 데이터
# youtube_df = pd.read_csv('/content/drive/My Drive/Final PJT - 업빛투/감성분석/data/youtube_naver_labeling.csv')

In [73]:
# youtube_df.ch_nm.unique()

### youtube 데이터 전체 로드

In [74]:
def youtube_db():

    com_list = {'하이닉스' : '000660',
            '현대차' : '005380',
            '삼성전자' : '005930',
            'LG화학' : '051910',
            '셀트리온' : '068270'}
    media_list = {'youtube_한국경제_':'hk_',
                'youtube_삼프로_':'sampro_',
                'youtube_슈카_':'suka_'}
    df_list = []
    for n,m in media_list.items():
        for i,j in com_list.items():

            db = pymysql.connect(
                user='root', 
                passwd='1234', 
                host='3.35.70.166', 
                db='proj', 
                charset = 'utf8'
            )

            cursor = db.cursor(pymysql.cursors.DictCursor)
            sql = "select * from youtube_"+ m + j +" where length(date) = 10"
            
            cursor.execute(sql)

            result = cursor.fetchall()

            # 데이터 프레임으로 변경
            globals() ['df_'+ m + j ] = pd.DataFrame(result)
            print('df_'+ m + j)
            df_list.append(globals() ['df_'+ m + j ])
            
            db.close()
    
    # print(df_list)
    df_total = pd.concat(df_list)
    
    df_total.rename(columns={'date': 'datetime'}, inplace=True)
    # DataFrame 형태를 통일하기 위해 date 컬럼 추가
    df_total['date'] = df_total['datetime'].str[0:4] + '-' + df_total['datetime'].str[4:6] + '-' + df_total['datetime'].str[6:8]
    df_total['date'] = pd.to_datetime(df_total['date'])

    # 시간순으로 정렬
    df_total.reset_index(inplace=True, drop=True)

    ### 4) Holidays
    db = pymysql.connect(user='root',
                         passwd='1234', 
                         host='3.35.70.166', 
                         db='proj',
                         charset='utf8')

    cursor = db.cursor(pymysql.cursors.DictCursor)

    # 4-1) 주말 및 공휴일 데이터
    sql = "select * from holidays"
    cursor.execute(sql)
    result = cursor.fetchall()

    holidays = pd.DataFrame(result)
    db.close()

    holidays['date'] = pd.to_datetime(holidays['date'])

    ## 2-1) 주말 및 공휴일만 추출
    market_closed =holidays[holidays['holiday']=="O"].reset_index(drop=True)
    
    ## 2-3) 휴장일 List 생성
    market_closed_list = list(market_closed['date'])
    
    ## 2-4) iteration limit 조정
    limit_number = 15000
    sys.setrecursionlimit(limit_number)
    
    
    while len(df_total[df_total['date'].isin(market_closed_list)]['date']) != 0:
        for hoil in df_total[df_total['date'].isin(market_closed_list)]['date'].index:
            df_total['date'][hoil] += datetime.timedelta(1)

    # 간단한 전처리
    df_total['text'] = df_total['text'].str.replace('\[|\]','')
    df_total['text'] = df_total['text'].str.replace('[a-z]','')
    df_total['text'] = df_total['text'].str.replace('[A-Z]','')
    df_total['text'] = df_total['text'].str.replace('으','')
    df_total['text'] = df_total['text'].str.replace('음악','')
    df_total['text'] = df_total['text'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
    df_total['text'] = df_total['text'].str.replace('^ +', "")

    return df_total

In [75]:
df_total = youtube_db()

df_hk_000660
df_hk_005380
df_hk_005930
df_hk_051910
df_hk_068270
df_sampro_000660
df_sampro_005380
df_sampro_005930
df_sampro_051910
df_sampro_068270
df_suka_000660
df_suka_005380
df_suka_005930
df_suka_051910
df_suka_068270


### 라벨링 패키지 및 모델 로딩

In [76]:
# 라벨 훈련 데이터
X_train = []
aa = pd.read_csv('/content/drive/My Drive/Final PJT - 업빛투/감성분석/youtube/train_label.csv').drop('Unnamed: 0',1)
aa.x = aa.x.str.replace('\[|\]','')
aa.x = aa.x.str.replace("'","")
aa.x = aa.x.str.replace(" ","")

for i in aa.x:
    temp = i.split(',')
    X_train.append(temp)

In [77]:
len(X_train)

145393

In [78]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
tokenizer = Tokenizer(19196)
tokenizer.fit_on_texts(X_train)
url = 'https://raw.githubusercontent.com/chaerui7967/stock_predict_news_and_youtube/master/Sentiment_Analysis/data/stopwords_ver1.txt'
stopwords = list(pd.read_csv(url, header=None)[0])

In [79]:
max_len = 200

In [80]:
# loaded_model = load_model('./best_model.h5')

In [81]:
loaded_model = load_model('/content/drive/My Drive/Final PJT - 업빛투/감성분석/youtube/best_model.h5')

In [82]:
def sentiment_predict(new_sentence):
    new_sentence = okt.morphs(new_sentence, stem=True) # 토큰화
    new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
    encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
    pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
    score = float(loaded_model.predict(pad_new)) # 예측

    if (score > 0.5):
        # print("{:.2f}% 확률로 긍정 리뷰\n".format(score * 100))
        return score, 1
    elif (score == 0.5):
        return score, 0
    else:
        # print("{:.2f}% 확률로 부정 리뷰\n".format((1 - score) * 100))
        return score, -1

In [83]:
# sentiment_predict('꿀잼') 연습

### 삼프로

In [84]:
sampro_df = df_total[(df_total.ch_nm == '삼프로TV_경제의신과함께') & (df_total.st_cd == '000660')]

In [85]:
sampro_df = sampro_df[sampro_df.date.isin(total_table.date)]

In [86]:
s_text = sampro_df.text.values

In [87]:
scores = [] # 긍정 확률
for i in s_text:
    score, _ = sentiment_predict(i)
    scores.append(score)

In [88]:
len(sampro_df),len(scores)

(27, 27)

In [89]:
sampro_df['sampro_youtube'] = np.array(scores).reshape(-1,1)
sampro_score = sampro_df.groupby(['date']).mean().reset_index()[['date','sampro_youtube']]

In [90]:
total_table = pd.merge(total_table, sampro_score, on='date',how='left')

In [91]:
total_table

,code,date,lstm,arima,fbprophet,RL,mail_news,mail_news_nsi,asia_news,asia_news_nsi,close,change,sampro_youtube
0,005930,2021-09-15,107611.110840,107531.570906,105170.103805,-1,1.000000,200.000000,0.409001,81.800144,107500,0.000000,0.385081
1,005930,2021-09-16,106315.300781,107757.076005,105230.196998,-1,0.577778,115.555556,0.789286,157.857143,104000,-0.032558,NaN
2,005930,2021-09-17,106507.104492,104246.157040,105085.161569,0,0.500000,100.000000,0.305641,61.128205,107000,0.028846,0.243515
3,005930,2021-09-23,105733.528809,106592.234904,104010.986018,-1,0.500000,100.333333,0.471347,94.519481,105500,-0.014019,0.461617
4,005930,2021-09-24,105307.092773,105347.857731,103765.708080,1,0.186667,37.333333,0.457738,91.547619,104000,-0.014218,0.079099
5,005930,2021-09-27,105843.469727,104174.314218,102621.668387,1,0.357143,71.428571,0.525683,105.367427,104500,0.004808,0.163172
6,005930,2021-09-28,104409.901855,104586.031889,102742.237966,-1,0.403175,80.801587,0.282062,56.537338,103500,-0.009569,0.326486
7,005930,2021-09-29,105048.020996,103492.603294,102328.026323,0,0.357143,71.428571,0.350000,70.000000,100000,-0.033816,0.284152
8,005930,2021-09-30,101771.489502,99993.489997,102223.087220,0,0.200000,40.000000,0.000000,0.000000,103000,0.030000,0.267341
9,005930,2021-10-01,101373.722412,102619.708155,101927.313323,0,0.500000,100.000000,0.200000,40.250000,100000,-0.029126,0.367717


### 한국경제

In [92]:
hk_df = df_total[(df_total.ch_nm == '한국경제TV') & (df_total.st_cd == '000660')]

In [93]:
hk_df = hk_df[hk_df.date.isin(total_table.date)]

In [94]:
hk_text = hk_df.text.values

scores = [] # 긍정 확률
for i in hk_text:
    score, _ = sentiment_predict(i)
    scores.append(score)

In [95]:
len(scores), len(hk_df)

(10, 10)

In [96]:
hk_df['hk_youtube'] = np.array(scores).reshape(-1,1)
hk_score = hk_df.groupby(['date']).mean().reset_index()[['date','hk_youtube']]

total_table = pd.merge(total_table, hk_score, on='date',how='left')

### 슈카월드

In [97]:
su_df = df_total[(df_total.ch_nm == '슈카월드') & (df_total.st_cd == '000660')]

su_df = su_df[su_df.date.isin(total_table.date)]

su_text = su_df.text.values

scores = [] # 긍정 확률
for i in su_text:
    score, _ = sentiment_predict(i)
    scores.append(score)

In [98]:
len(su_df),len(scores)

(1, 1)

In [99]:
su_df['suka_youtube'] = np.array(scores).reshape(-1,1)
suka_score = su_df.groupby(['date']).mean().reset_index()[['date','suka_youtube']]

total_table = pd.merge(total_table, suka_score, on='date',how='left')

In [100]:
total_table.columns

Index(['code', 'date', 'lstm', 'arima', 'fbprophet', 'RL', 'mail_news',
       'mail_news_nsi', 'asia_news', 'asia_news_nsi', 'close', 'change',
       'sampro_youtube', 'hk_youtube', 'suka_youtube'],
      dtype='object')

In [101]:
total_table = total_table[['code', 'date', 'lstm', 'arima', 'fbprophet', 'RL', 'mail_news',
       'mail_news_nsi', 'asia_news', 'asia_news_nsi',
       'sampro_youtube', 'hk_youtube', 'suka_youtube', 'close', 'change']]

# 최종결과

In [102]:
total_table

,code,date,lstm,arima,fbprophet,RL,mail_news,mail_news_nsi,asia_news,asia_news_nsi,sampro_youtube,hk_youtube,suka_youtube,close,change
0,005930,2021-09-15,107611.110840,107531.570906,105170.103805,-1,1.000000,200.000000,0.409001,81.800144,0.385081,NaN,0.50873,107500,0.000000
1,005930,2021-09-16,106315.300781,107757.076005,105230.196998,-1,0.577778,115.555556,0.789286,157.857143,NaN,0.421872,NaN,104000,-0.032558
2,005930,2021-09-17,106507.104492,104246.157040,105085.161569,0,0.500000,100.000000,0.305641,61.128205,0.243515,0.535122,NaN,107000,0.028846
3,005930,2021-09-23,105733.528809,106592.234904,104010.986018,-1,0.500000,100.333333,0.471347,94.519481,0.461617,0.397960,NaN,105500,-0.014019
4,005930,2021-09-24,105307.092773,105347.857731,103765.708080,1,0.186667,37.333333,0.457738,91.547619,0.079099,0.365010,NaN,104000,-0.014218
5,005930,2021-09-27,105843.469727,104174.314218,102621.668387,1,0.357143,71.428571,0.525683,105.367427,0.163172,NaN,NaN,104500,0.004808
6,005930,2021-09-28,104409.901855,104586.031889,102742.237966,-1,0.403175,80.801587,0.282062,56.537338,0.326486,NaN,NaN,103500,-0.009569
7,005930,2021-09-29,105048.020996,103492.603294,102328.026323,0,0.357143,71.428571,0.350000,70.000000,0.284152,NaN,NaN,100000,-0.033816
8,005930,2021-09-30,101771.489502,99993.489997,102223.087220,0,0.200000,40.000000,0.000000,0.000000,0.267341,0.495956,NaN,103000,0.030000
9,005930,2021-10-01,101373.722412,102619.708155,101927.313323,0,0.500000,100.000000,0.200000,40.250000,0.367717,0.965788,NaN,100000,-0.029126


In [103]:
while total_table.iloc[-1:].isnull().sum().sum() != 0:
    if total_table.iloc[:1].isnull().sum().sum() != 0:
      for i in range(1, len(total_table)):
        total_table = total_table.fillna(method='pad')
      break
    total_table = total_table.fillna(method='pad')

In [104]:
total_table

,code,date,lstm,arima,fbprophet,RL,mail_news,mail_news_nsi,asia_news,asia_news_nsi,sampro_youtube,hk_youtube,suka_youtube,close,change
0,005930,2021-09-15,107611.110840,107531.570906,105170.103805,-1,1.000000,200.000000,0.409001,81.800144,0.385081,NaN,0.50873,107500,0.000000
1,005930,2021-09-16,106315.300781,107757.076005,105230.196998,-1,0.577778,115.555556,0.789286,157.857143,0.385081,0.421872,0.50873,104000,-0.032558
2,005930,2021-09-17,106507.104492,104246.157040,105085.161569,0,0.500000,100.000000,0.305641,61.128205,0.243515,0.535122,0.50873,107000,0.028846
3,005930,2021-09-23,105733.528809,106592.234904,104010.986018,-1,0.500000,100.333333,0.471347,94.519481,0.461617,0.397960,0.50873,105500,-0.014019
4,005930,2021-09-24,105307.092773,105347.857731,103765.708080,1,0.186667,37.333333,0.457738,91.547619,0.079099,0.365010,0.50873,104000,-0.014218
5,005930,2021-09-27,105843.469727,104174.314218,102621.668387,1,0.357143,71.428571,0.525683,105.367427,0.163172,0.365010,0.50873,104500,0.004808
6,005930,2021-09-28,104409.901855,104586.031889,102742.237966,-1,0.403175,80.801587,0.282062,56.537338,0.326486,0.365010,0.50873,103500,-0.009569
7,005930,2021-09-29,105048.020996,103492.603294,102328.026323,0,0.357143,71.428571,0.350000,70.000000,0.284152,0.365010,0.50873,100000,-0.033816
8,005930,2021-09-30,101771.489502,99993.489997,102223.087220,0,0.200000,40.000000,0.000000,0.000000,0.267341,0.495956,0.50873,103000,0.030000
9,005930,2021-10-01,101373.722412,102619.708155,101927.313323,0,0.500000,100.000000,0.200000,40.250000,0.367717,0.965788,0.50873,100000,-0.029126


In [105]:
total_table.to_csv('./total_table.csv', index=False)

In [106]:
total_table = pd.read_csv('./total_table.csv')

In [107]:
total_table_ud = np.round(total_table,2)


for i in range(len(total_table_ud)-1,0,-1):
  if total_table_ud['lstm'].iloc[i] > total_table_ud['lstm'].iloc[i-1]:
    total_table_ud['lstm'].iloc[i] = 1
  else:
    total_table_ud['lstm'].iloc[i] = -1
  
  if total_table_ud['arima'].iloc[i] > total_table_ud['close'].iloc[i-1]:
    total_table_ud['arima'].iloc[i] = 1
  else:
    total_table_ud['arima'].iloc[i] = -1
  
  if total_table_ud['fbprophet'].iloc[i] > total_table_ud['fbprophet'].iloc[i-1]:
    total_table_ud['fbprophet'].iloc[i] = 1
  else:
    total_table_ud['fbprophet'].iloc[i] = -1
  
total_table_ud['mail_news'] = np.where(total_table_ud['mail_news']>0.5,1,
                                       np.where(total_table_ud['mail_news']<0.5,-1,0))

total_table_ud['mail_news_nsi'] = np.where(total_table_ud['mail_news_nsi']>100,1,
                                       np.where(total_table_ud['mail_news_nsi']<100,-1,0))

total_table_ud['asia_news'] = np.where(total_table_ud['asia_news']>0.5,1,
                                       np.where(total_table_ud['asia_news']<0.5,-1,0))

total_table_ud['asia_news_nsi'] = np.where(total_table_ud['asia_news_nsi']>100,1,
                                       np.where(total_table_ud['asia_news_nsi']<100,-1,0))

total_table_ud['sampro_youtube'] = np.where(total_table_ud['sampro_youtube']>0.5,1,
                                       np.where(total_table_ud['sampro_youtube']<0.5,-1,0))

total_table_ud['hk_youtube'] = np.where(total_table_ud['hk_youtube']>0.5,1,
                                       np.where(total_table_ud['hk_youtube']<0.5,-1,0))

total_table_ud['suka_youtube'] = np.where(total_table_ud['suka_youtube']>0.5,1,
                                       np.where(total_table_ud['suka_youtube']<0.5,-1,0))

total_table_ud['change'] = np.where(total_table_ud['change']>0,1,
                                       np.where(total_table_ud['change']<0,-1,0))
total_table_ud.drop('close',1,inplace=True)

In [108]:
total_table_ud

,code,date,lstm,arima,fbprophet,RL,mail_news,mail_news_nsi,asia_news,asia_news_nsi,sampro_youtube,hk_youtube,suka_youtube,change
0,5930,2021-09-15,107611.11,107531.57,105170.1,-1,1,1,-1,-1,-1,0,1,0
1,5930,2021-09-16,-1.00,1.00,1.0,-1,1,1,1,1,-1,-1,1,-1
2,5930,2021-09-17,1.00,1.00,-1.0,0,0,0,-1,-1,-1,1,1,1
3,5930,2021-09-23,-1.00,-1.00,-1.0,-1,0,1,-1,-1,-1,-1,1,-1
4,5930,2021-09-24,-1.00,-1.00,-1.0,1,-1,-1,-1,-1,-1,-1,1,-1
5,5930,2021-09-27,1.00,1.00,-1.0,1,-1,-1,1,1,-1,-1,1,0
6,5930,2021-09-28,-1.00,1.00,1.0,-1,-1,-1,-1,-1,-1,-1,1,-1
7,5930,2021-09-29,1.00,-1.00,-1.0,0,-1,-1,-1,-1,-1,-1,1,-1
8,5930,2021-09-30,-1.00,-1.00,-1.0,0,-1,-1,-1,-1,-1,0,1,1
9,5930,2021-10-01,-1.00,-1.00,-1.0,0,0,0,-1,-1,-1,1,1,-1


In [110]:
total_table_ud.to_csv('./hy_table_ud.csv', index=False)